In [1]:
import requests
import pandas as pd
import io

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

In [ ]:
sys.path.append('../src/general_cleaning.py')
from general_cleaning import *

In [ ]:
sys.path.append('../src/api_cleaning.py')
from api_cleaning import *

In [ ]:
sys.path.append('../src/kaggle_cleaning.py')
from kaggle_cleaning import *

## Data from Kaggle

### Suicide-rates-overview-1985-to-2016

In [ ]:
!kaggle datasets download -d russellyates88/suicide-rates-overview-1985-to-2016

In [ ]:
!tar -xzvf suicide-rates-overview-1985-to-2016.zip

In [ ]:
!mv ../notebooks/master.csv ../input/master.csv

In [ ]:
kaggle_data = pd.read_csv("../input/master.csv")

## Data from CDC API

### NCHS - Leading Causes of Death: United States

In [ ]:
url = "https://data.cdc.gov/resource/bi63-dtpu.csv"

In [ ]:
response = requests.get(url)

In [ ]:
response.status_code

In [ ]:
response_text = response.text

In [ ]:
api_data = pd.read_csv(io.StringIO(response_text))

## Cleaning both datasets

### Kaggle dataset

In [ ]:
kaggle_data

In [ ]:
# Lista de columnas a eliminar
columns_to_drop = ['population', 'deaths/100k_pop', 'country-year', 'HDI for year', ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation']
K1 = droping_columns(kaggle_data, columns_to_drop)

In [ ]:
# Dataframe que solo contiene los datos de USA
K2 = only_usa(K1)

In [ ]:
# Lista de los años con los que me quiero quedar 
wanted_years = range(2000, 2016)
L1 = looking_for_elements(K2.year, wanted_years)

In [ ]:
# Lista USA
L2 = new_country(wanted_years)

In [ ]:
# Lista de la suma de muertes por año
L3 = new_deaths(K2, wanted_years)

In [ ]:
K3 = df_from_3lst(L1, L2,L3)

In [ ]:
names_dict = {'list1':'Year', 'list2':'Country', 'list3':'Deaths'}
K4 = rename_columns(K3, names_dict)

In [ ]:
# K4.to_csv('../output/kaggle_dataset.csv')

### API dataset

In [ ]:
api_data

In [ ]:
# Lista de columnas a eliminar
to_drop = ['_113_cause_name', 'aadr']
A1 =  droping_columns(api_data, to_drop)

In [ ]:
# Lista de los estados con los que me quiero quedar 
wanted_states = ['New York', 'California', 'Maryland', 'Connecticut', 'Massachuets', 'District of Columbia', 'Illinois', 'Rhode Island', 'New Jersey']
C1 = looking_for_elements(A1.state, wanted_states)

In [ ]:
# Lista de los años con los que me quiero quedar 
wanted_years = range(2000, 2016)
C2 = looking_for_elements(A1.year, wanted_years)

In [ ]:
# Lista de causas de muerte con las que me quiero quedar 
wanted_death = ['Suicide']
C3 = looking_for_elements(A1.cause_name, wanted_death)

In [ ]:
# Dataframe final solo con algunos estados
A2 = only_states(A1, C1, C2, C3)

In [ ]:
# Nuevo indice
A3 = new_index(A2)

In [ ]:
new_names_dict = {'year':'Year', 'cause_name':'Death_Cause', 'state':'State', 'deaths':'Deaths'}
A4 = rename_columns(A3, new_names_dict)

In [ ]:
# A4.to_csv('../output/api_dataset.csv')

## Issues

Mis datos van a ir desde el año 2000 hasta el año 2015, me quedará el último de año de gobierno, 2016, no pasa nada.